<a href="https://colab.research.google.com/github/Sansith/gpt2sp/blob/ensemble-gpt2/model_training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training Script

### Necessary Library

In [10]:
pip install torch pandas===1.5.3 transformers numpy tokenizers koila tensorboard Cython

In [11]:

import pdb

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
cd drive/MyDrive/Year4/FYP/effort-estimation/gpt2sp

[Errno 2] No such file or directory: 'drive/MyDrive/Year4/FYP/effort-estimation/gpt2sp'
/content


In [19]:
cd drive/MyDrive/gpt2sp

/content/drive/.shortcut-targets-by-id/1YObNZjwdIaLufXlBhNXVVkPCz4YQrBTV/gpt2sp


In [20]:


ls

abe0/                               models/
all_tokenizers/                     model_training_notebook_bigBird.ipynb
base_model_training_notebook.ipynb  model_training_notebook.ipynb
BigBird.py                          model_training_notebook_new_impl.ipynb
corpus_tokenization_comparison/     __pycache__/
custom_transformers_interpret/      README.md
data_model_analysis/                results/
GPT2SPEN.py                         sp_dataset/
GPT2SP_inspection_notebook.ipynb    tb/
GPT2SP.py                           tokenizer_training_notebook.ipynb
LICENSE                             vocab_and_tokenization_comparison.ipynb
logo/                               xai_tokens/


In [21]:
from transformers.modeling_outputs import SequenceClassifierOutputWithPast
import torch.nn as nn
from transformers import GPT2Model, GPT2PreTrainedModel
import torch


class GPT2SPEN(GPT2PreTrainedModel):
    _keys_to_ignore_on_load_missing = [r"h\.\d+\.attn\.masked_bias", r"lm_head\.weight"]

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.transformer = GPT2Model(config)
        self.transformer_description = GPT2Model(config)
        print("n embd :: ",config.n_embd)
        self.dense1 = nn.Linear(2* config.n_embd, 8 * config.n_embd, bias=False)
        self.dense2 = nn.Linear(8 * config.n_embd,2* config.n_embd, bias=False)
        self.score = nn.Linear(2* config.n_embd, self.num_labels, bias=False)

        self.init_weights()

        # Model parallel
        self.model_parallel = False
        self.device_map = None


    def forward(
        self,
        input_ids_title=None,
        input_ids_description=None,
        past_key_values=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        use_cache=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`):
            Labels for computing the sequence classification/regression loss. Indices should be in :obj:`[0, ...,
            config.num_labels - 1]`. If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # Title model
        transformer_outputs_title = self.transformer(
            input_ids_title,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        title_hidden_states = transformer_outputs_title[0]


        # Description model
        transformer_outputs_description = self.transformer_description(
            input_ids_description,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        description_hidden_states = transformer_outputs_description[0]

        # Concatenate the output tensors from both models
        concatenated_hidden_states = torch.cat((title_hidden_states, description_hidden_states), dim=2)
        # print("concatenated_hidden_states len :: ",len(concatenated_hidden_states))
        # MLP Layer
        # pdb.set_trace()
        hidden_states = self.dense1(concatenated_hidden_states)
        hidden_states = self.dense2(hidden_states)

        logits = self.score(hidden_states)

        if input_ids_title is not None:
            batch_size, sequence_length = input_ids_title.shape[:2]
        else:
            batch_size, sequence_length = inputs_embeds.shape[:2]

        assert (
            self.config.pad_token_id is not None or batch_size == 1
        ), "Cannot handle batch sizes > 1 if no padding token is defined."

        if self.config.pad_token_id is None:
            sequence_lengths = -1
        else:
            if input_ids_title is not None:
                sequence_lengths = torch.ne(input_ids_title, self.config.pad_token_id).sum(-1) - 1
            else:
                sequence_lengths = -1
                logger.warning(
                    f"{self.__class__.__name__} will not detect padding tokens in `inputs_embeds`. Results may be "
                    f"unexpected if using padding tokens in conjunction with `inputs_embeds.`"
                )

        pooled_logits = logits[range(batch_size), sequence_lengths]

        loss = None
        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = nn.L1Loss()
                # pdb.set_trace()
                loss = loss_fct(pooled_logits.view(-1), labels.to(self.dtype).view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(pooled_logits.view(-1, self.num_labels), labels.view(-1))

        if not return_dict:
            output = (logits,) + transformer_outputs_title[1:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutputWithPast(
            loss=loss,
            logits=logits,
            past_key_values=transformer_outputs_title.past_key_values,
            hidden_states=transformer_outputs_title.hidden_states,
            attentions=transformer_outputs_title.attentions,
        )

In [22]:
import torch
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
import numpy as np
import time
from torch.utils.tensorboard import SummaryWriter
from GPT2SP import GPT2ForSequenceClassification as GPT2SP
#from GPT2SPEN import GPT2ForSequenceClassification as GPT2SPEN
from transformers import GPT2ForSequenceClassification as LinearGPT2
from transformers import GPT2Config
import os
from tokenizers import Tokenizer
import torch.nn as nn

### Hyperparameters

In [23]:
global EPOCHS, BATCH_SIZE_RATIO, SEQUENCE_LEN, LEARNING_RATE, TOKENIZER, MODEL_NAME , ADD_DESCRIPTION

EPOCHS = 20
BATCH_SIZE_RATIO = 0.1# within proj: 0.3 / cross proj: 0.4
SEQUENCE_LEN = 20
LEARNING_RATE = 5e-4
TOKENIZER = 'gpt2' # available: gpt2, wordlevel, sentencepiece, wordpiece
MODEL_NAME = 'gpt2spen' # available:gpt2spen , gpt2sp, gpt2
ADD_DESCRIPTION = False

# define device
global DEVICE
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# define files to be used
global DATA_PATH
DATA_PATH = './sp_dataset/marked_data/'

### Static Methods and Variables

In [24]:
OUTPUT = '  '
MODEL = None
DYNAMIC_BATCH = True
BATCH_SIZE = None
WITHIN_PROJECT = None
MAE_RECORDS = []
MDAE_RECORDS = []

def data_processing(file_pair):
    global BATCH_SIZE, BATCH_SIZE_RATIO, DATA_PATH, WITHIN_PROJECT, DYNAMIC_BATCH

    train_data = pd.DataFrame(columns=['title', 'description',"label"])
    for train_file_name in file_pair['train']:
        fname = DATA_PATH + train_file_name + '.csv'
        df = prepare_dataframe(fname)
        train_data = train_data.append(df)

    # data split
    if WITHIN_PROJECT:
        train_title,train_description,train_labels,  val_title,val_description,val_labels,  test_title,test_description,test_labels = within_project_split(train_data)
    else:
        train_title,train_description , train_labels, val_title,val_description , val_labels = train_val_split(train_data, 0.6)
    # define batch size dynamically based on training length
    if DYNAMIC_BATCH:
        BATCH_SIZE = int(len(train_title) * BATCH_SIZE_RATIO)
    # tokenization
    title_tokens_train = tokenization(train_title.tolist())
    description_tokens_train = tokenization(train_description.tolist())

    title_tokens_val = tokenization(val_title.tolist())
    description_tokens_val = tokenization(val_description.tolist())


    train_seq_titles = torch.tensor(title_tokens_train['input_ids'])
    train_seq_descriptions = torch.tensor(description_tokens_train['input_ids'])
    train_y = torch.tensor(train_labels.tolist()).type(torch.LongTensor)

    train_dataloader = prepare_dataloader(train_seq_titles,train_seq_descriptions ,  train_y, sampler_type='random')


    val_seq_titles = torch.tensor(title_tokens_val['input_ids'])
    val_seq__descriptions = torch.tensor(description_tokens_val['input_ids'])
    val_y = torch.tensor(val_labels.tolist()).type(torch.LongTensor)

    val_dataloader = prepare_dataloader(val_seq_titles,val_seq__descriptions ,val_y, sampler_type='sequential')

    # prepare testing datasets
    all_test_dataloader = []
    test_file_names = []
    if WITHIN_PROJECT:
        tokens_test_title = tokenization(test_title.tolist())
        tokens_test_description = tokenization(test_description.tolist())

        test_seq_title = torch.tensor(tokens_test_title['input_ids'])
        test_seq_description = torch.tensor(tokens_test_description['input_ids'])
        test_y = torch.tensor(test_labels.tolist()).type(torch.LongTensor)

        test_dataloader = prepare_dataloader(test_seq_title,test_seq_description, test_y, sampler_type='sequential')
        all_test_dataloader.append(test_dataloader)
        test_file_names.append(file_pair['test'][0])
        return file_pair, train_dataloader, val_dataloader, all_test_dataloader, test_file_names

    for test_file_name in file_pair['test']:
        fname = DATA_PATH + test_file_name + '.csv'
        test_data = prepare_dataframe(fname)

        test_title = test_data['title']
        test_description = test_data['description']
        test_labels = test_data['label']

        # tokenization
        tokens_test_title = tokenization(test_title.tolist())
        tokens_test_description = tokenization(test_description.tolist())
        test_seq_title = torch.tensor(tokens_test_title['input_ids'])
        test_seq_description = torch.tensor(tokens_test_description['input_ids'])

        test_y = torch.tensor(test_labels.tolist()).type(torch.LongTensor)
        test_dataloader = prepare_dataloader(test_seq_title,test_seq_description, test_y, sampler_type='sequential')

        all_test_dataloader.append(test_dataloader)
        test_file_names.append(test_file_name)
    print('cross project data processing!')
    return file_pair, train_dataloader, val_dataloader, all_test_dataloader, test_file_names


def train_val_split(data, split_ratio):
    print('cross project split!')
    split_point = int(len(data) * split_ratio)

    train_title = data['title'][:split_point]
    train_description = data['description'][:split_point]
    train_labels = data['label'][:split_point]

    val_title = data['title'][split_point:]
    val_description = data['description'][split_point:]
    val_labels = data['label'][split_point:]
    return train_title,train_description , train_labels, val_title,val_description , val_labels


def tokenization(text_list):
    global TOKENIZER, SEQUENCE_LEN, MODEL
    # tokenization
    if TOKENIZER == 'wordpiece':
        print('using wordpiece tokenizer!')
        tokenizer = BertTokenizer('all_tokenizers/word_piece/vocab.txt')
    elif TOKENIZER == 'sentencepiece':
        print('using sentencepiece tokenizer!')
        tokenizer = XLNetTokenizer('all_tokenizers/sentence_piece/spm_tokenizer.model', padding_side='right')
    elif TOKENIZER == 'wordlevel':
        print('using wordlevel tokenizer!')
        tokenizer = Tokenizer.from_file('all_tokenizers/word_level/wordlevel.json')
        encoded_sentences = {'input_ids':[]}
        for sentence in text_list:
            encoded = tokenizer.encode(sentence)
            encoded = encoded.ids
            if len(encoded) > SEQUENCE_LEN:
                encoded = encoded[:SEQUENCE_LEN]
            elif len(encoded) < SEQUENCE_LEN:
                padding = SEQUENCE_LEN - len(encoded)
                for _ in range(padding):
                    encoded.append(3)
            encoded_sentences['input_ids'].append(encoded)
        return encoded_sentences
    elif TOKENIZER == 'gpt2':
        print('using pretrained gpt-2 tokenizer')
        tokenizer = GPT2Tokenizer.from_pretrained(TOKENIZER)
        tokenizer.pad_token = '[PAD]'
    return tokenizer.batch_encode_plus(text_list, truncation=True, max_length=SEQUENCE_LEN, padding='max_length')


def prepare_dataframe(file_name):
    data = pd.read_csv(file_name)
    # some rows have no description, fill blank to avoid Null
    data = data.fillna(' ')


    # if ADD_DESCRIPTION :
    #   print("### text : title+description")
    #   d = {'text': (data['title'] + " : " + data["description"]).tolist(), 'label': data['storypoint']}
    # else:
    #   print("### text : title")
    #   d = {'text': (data['title']).tolist(), 'label': data['storypoint']}

    d = { 'title':(data['title']).tolist(), 'description': (data["description"]).tolist() , 'label': data['storypoint'] }
    print("Input data feed ::: ",d.keys())
    return pd.DataFrame(data=d)


def prepare_dataloader(seq_title,seq_description, y, sampler_type):
    global BATCH_SIZE
    tensor_dataset = TensorDataset(seq_title,seq_description, y)
    if sampler_type == 'random':
        sampler = RandomSampler(tensor_dataset)
    elif sampler_type == 'sequential':
        sampler = SequentialSampler(tensor_dataset)
    print("BATCH_SIZE : ",BATCH_SIZE)
    dataloader = DataLoader(tensor_dataset, sampler=sampler, batch_size=BATCH_SIZE)
    return dataloader


def within_project_split(data):
    print('within project split!')

    train_val_split_point = int(len(data) * 0.6)
    val_test_split_point = int(len(data) * 0.8)

    train_title = data['title'][:train_val_split_point]
    train_description = data['description'][:train_val_split_point]
    train_labels = data['label'][:train_val_split_point]

    val_title = data['title'][train_val_split_point:val_test_split_point]
    val_description = data['description'][train_val_split_point:val_test_split_point]
    val_labels = data['label'][train_val_split_point:val_test_split_point]

    test_title = data['title'][val_test_split_point:]
    test_description = data['description'][val_test_split_point:]
    test_labels = data['label'][val_test_split_point:]

    return train_title,train_description,train_labels,  val_title,val_description,val_labels,  test_title,test_description,test_labels


def train_eval_test(file_pair, train_dataloader, val_dataloader, all_test_dataloader, model, test_file_names):
    global LEARNING_RATE, EPOCHS, MAE_RECORDS, MDAE_RECORDS, DEVICE

    # Optimizerrr -->
    optimizer = AdamW(MODEL.parameters(), lr=LEARNING_RATE)
    # Total number of training steps is [number of batches] x [number of epochs]
    total_steps = len(train_dataloader) * EPOCHS
    # Create the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
    print("Start training for ", file_pair, ".....")
    training_start_time = time.time()

    # tensorboard writer
    writer_path = 'tb/' + str(file_pair['train'][0]) + '_' + str(file_pair['test'][0])
    writer = SummaryWriter(writer_path)

    # vars for model selection
    min_eval_loss_epoch = [10000, 0]

    time_records = []
    MAE_RECORDS = []
    MDAE_RECORDS = []
    start_time = time.time()
    loss_fct = nn.L1Loss()
    for e in range(EPOCHS):
        # ---TRAINING---
        # clean GPU memory
        torch.cuda.empty_cache()
        print(">>> epoch ", e)
        # set model into train mode
        model.train()
        total_train_loss = 0

        for step, batch in enumerate(train_dataloader):
            # pdb.set_trace()
            b_input_ids_title = batch[0].to(DEVICE)
            b_input_ids_description = batch[1].to(DEVICE)
            b_labels = batch[2].to(DEVICE)

            model.zero_grad()

            result = model(input_ids_title=b_input_ids_title,
                           input_ids_description=b_input_ids_description,
                           labels=b_labels,
                           return_dict=True)
            loss = result.loss
            logits = result.logits
            total_train_loss += loss.item()
            # Calculates the gradients
            loss.backward()
            # The clip_grad_norm_ function clips (limits) the norm (magnitude) of the gradients to a maximum value specified by the user.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            #updates the weights and bias accrding to the calculated gradients
            optimizer.step()
            # update learning rates
            scheduler.step()
            # clean memory
            del step, batch, b_input_ids_title,b_input_ids_description, b_labels, result, loss, logits

        avg_train_loss = total_train_loss / len(train_dataloader)
        print(" Average training MAE loss: {0:.2f}".format(avg_train_loss))
        writer.add_scalar('loss/train', avg_train_loss, e)
        # clean memory
        del avg_train_loss, total_train_loss

        time_records.append(time.time() - start_time)

        # ---EVAL---
        print("-")
        # set model into eval mode
        model.eval()
        total_eval_loss = 0
        for batch in val_dataloader:
            b_input_ids_title = batch[0].to(DEVICE)
            b_input_ids_description = batch[1].to(DEVICE)
            b_labels = batch[2].to(DEVICE)

            model.zero_grad()
            result = model(input_ids_title=b_input_ids_title,
                           input_ids_description=b_input_ids_description,
                           labels=b_labels,
                           return_dict=True)
            loss = result.loss
            logits = result.logits
            total_eval_loss += loss.item()
            # clean memory
            del b_input_ids_title,b_input_ids_description, b_labels, batch, result, loss, logits
        avg_eval_loss = total_eval_loss / len(val_dataloader)
        print(" Average eval MAE loss: {0:.2f}".format(avg_eval_loss))

        if avg_eval_loss <= min_eval_loss_epoch[0]:
            min_eval_loss_epoch[0] = avg_eval_loss
            min_eval_loss_epoch[1] = e

        writer.add_scalar('loss/eval', avg_eval_loss, e)
        # clean memory
        del avg_eval_loss, total_eval_loss
        # save model state to dict
        torch.save(model.state_dict(), './models/' + 'epo_' + str(e))

        print("===============================")

        # testing on holdout data
        index = 0
        for test_dataloader in all_test_dataloader:
            test_file_name = test_file_names[index]
            index += 1
            testing_start_time = time.time()
            predictions = []
            true_labels = []
            for batch in test_dataloader:
                # batch = tuple(t.to(DEVICE) for t in batch)
                b_input_ids_title = batch[0].to(DEVICE)
                b_input_ids_description = batch[1].to(DEVICE)
                b_labels = batch[2].to(DEVICE)

                # b_input_ids, b_labels = batch
                with torch.no_grad():
                    logits = model(input_ids_title=b_input_ids_title,input_ids_description=b_input_ids_description)
                logits = logits['logits'].detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                predictions.append(logits)
                true_labels.append(label_ids)

                del b_input_ids_title, b_input_ids_description ,b_labels
            # calculate errors
            distance_records = []
            for i in range(len(predictions)):
                for j in range(len(predictions[i])):
                    distance = abs(predictions[i][j] - true_labels[i][j])
                    distance_records.append(distance)

            ## MAE = mean value of all absolute errors (stored in distance_records)
            MAE = np.mean(np.array(distance_records))
            ## MdAE = median value of all absolute errors (stored in distance_records)
            MdAE = np.median(np.array(distance_records))

            MAE_RECORDS.append(MAE)
            MDAE_RECORDS.append(MdAE)

            global OUTPUT
            OUTPUT +=  'Epochs ' + str(e) + '\n'
            OUTPUT += 'MAE: ' + str(MAE) + '\n'
            OUTPUT += 'MdAE: ' + str(MdAE) + '\n\n'
            print('MAE: ', MAE)
            print('MdAE: ', MdAE)
    writer.flush()
    writer.close()

    # select model
    os.rename('models/epo_' + str(min_eval_loss_epoch[1]),
              'models/' + str(file_pair['train'][0]) + '_'
              + str(file_pair['test'][0]) + '_epo_' + str(min_eval_loss_epoch[1]))

    # del unwanted models
    for i in range(20):
        try:
            os.remove("models/epo_" + str(i))
        except:
            continue

    OUTPUT += 'MAE: ' + str(MAE_RECORDS[min_eval_loss_epoch[1]]) \
                + '  MdAE: ' + str(MDAE_RECORDS[min_eval_loss_epoch[1]]) + '\n'
    OUTPUT += 'training time: ' + str(time_records[min_eval_loss_epoch[1]]) + '\n'
    OUTPUT += 'Epochs: ' + str(min_eval_loss_epoch[1]) +'\n'
    global BATCH_SIZE
    OUTPUT += 'batch size: ' + str(BATCH_SIZE) + '\n'
    global ADD_DESCRIPTION
    OUTPUT += 'Description added : ' + str(ADD_DESCRIPTION) + '\n'


    print('all done for one project')

### Within Project Training Script

In [25]:
torch.cuda.empty_cache()

In [26]:
global WITHIN_PROJECT
WITHIN_PROJECT = True

TRAIN_TEST_FILE_PAIRS = [
                        # {'train': ['appceleratorstudio'], 'test': ['appceleratorstudio']},
                        # {'train': ['aptanastudio'], 'test': ['aptanastudio']},
                        # {'train': ['bamboo'], 'test': ['bamboo']},
                        # {'train': ['clover'], 'test': ['clover']},
                        # {'train': ['datamanagement'], 'test': ['datamanagement']},
                        {'train': ['duracloud'], 'test': ['duracloud']},
                        {'train': ['jirasoftware'], 'test': ['jirasoftware']},
                        {'train': ['mesos'], 'test': ['mesos']},
                        {'train': ['moodle'], 'test': ['moodle']},
                        {'train': ['mule'], 'test': ['mule']},
                        {'train': ['mulestudio'], 'test': ['mulestudio']},
                        # {'train': ['springxd'], 'test': ['springxd']},
                        # {'train': ['talenddataquality'], 'test': ['talenddataquality']},
                        # {'train': ['talendesb'], 'test': ['talendesb']},
                        # {'train': ['titanium'], 'test': ['titanium']},
                        # {'train': ['usergrid'], 'test': ['usergrid']},
                        ]


def main():
    global TRAIN_TEST_FILE_PAIRS, MODEL, TOKENIZER, MODEL_NAME
    for file in TRAIN_TEST_FILE_PAIRS:
        if TOKENIZER == 'bbpe':
            config = GPT2Config(num_labels=1, pad_token_id=50257)
        elif TOKENIZER == 'gpt2':
            config = GPT2Config(num_labels=1, pad_token_id=50256)
        elif TOKENIZER == 'wordpiece':
            config = GPT2Config(num_labels=1, pad_token_id=0)
        elif TOKENIZER == 'sentencepiece':
            config = GPT2Config(num_labels=1, pad_token_id=0)
        elif TOKENIZER == 'wordlevel':
            config = GPT2Config(num_labels=1, pad_token_id=3)
        if MODEL_NAME == 'gpt2':
            MODEL = LinearGPT2.from_pretrained('gpt2', config=config)
            if torch.cuda.is_available() :
              MODEL.cuda()
        elif MODEL_NAME == 'gpt2sp':
            MODEL = GPT2SP.from_pretrained('gpt2', config=config)
            if torch.cuda.is_available():
              MODEL.cuda()
        elif MODEL_NAME == 'gpt2spen':
            MODEL = GPT2SPEN.from_pretrained('gpt2', config=config)
            if torch.cuda.is_available():
              MODEL.cuda()



        file_pair, train_dataloader, val_dataloader, all_test_dataloader, test_file_names = data_processing(file_pair=file)
        train_eval_test(file_pair, train_dataloader, val_dataloader, all_test_dataloader, MODEL, test_file_names)
        del MODEL
        torch.cuda.empty_cache()
        global OUTPUT
        with open('./results/' + str(file['train'][0]) + '_' + str(file['test'][0]) +'.txt', 'w+') as f:
            f.writelines(OUTPUT)
            print('results have been written into a text file!')
            OUTPUT = ""


if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

n embd ::  768


Some weights of GPT2SPEN were not initialized from the model checkpoint at gpt2 and are newly initialized: ['dense1.weight', 'dense2.weight', 'score.weight', 'transformer_description.h.0.attn.c_attn.bias', 'transformer_description.h.0.attn.c_attn.weight', 'transformer_description.h.0.attn.c_proj.bias', 'transformer_description.h.0.attn.c_proj.weight', 'transformer_description.h.0.ln_1.bias', 'transformer_description.h.0.ln_1.weight', 'transformer_description.h.0.ln_2.bias', 'transformer_description.h.0.ln_2.weight', 'transformer_description.h.0.mlp.c_fc.bias', 'transformer_description.h.0.mlp.c_fc.weight', 'transformer_description.h.0.mlp.c_proj.bias', 'transformer_description.h.0.mlp.c_proj.weight', 'transformer_description.h.1.attn.c_attn.bias', 'transformer_description.h.1.attn.c_attn.weight', 'transformer_description.h.1.attn.c_proj.bias', 'transformer_description.h.1.attn.c_proj.weight', 'transformer_description.h.1.ln_1.bias', 'transformer_description.h.1.ln_1.weight', 'transform

Input data feed :::  dict_keys(['title', 'description', 'label'])
within project split!
using pretrained gpt-2 tokenizer


...

<ipython-input-24-55aa6fcdda15>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(df)


using pretrained gpt-2 tokenizer
using pretrained gpt-2 tokenizer
using pretrained gpt-2 tokenizer
BATCH_SIZE :  100
BATCH_SIZE :  100
using pretrained gpt-2 tokenizer
using pretrained gpt-2 tokenizer
BATCH_SIZE :  100
Start training for  {'train': ['mesos'], 'test': ['mesos']} .....
>>> epoch  0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


 Average training MAE loss: 64.03
-
 Average eval MAE loss: 17.77
MAE:  16.045145
MdAE:  16.186935
>>> epoch  1
 Average training MAE loss: 8.93
-
 Average eval MAE loss: 2.37
MAE:  2.634709
MdAE:  2.6862524
>>> epoch  2
 Average training MAE loss: 3.27
-
 Average eval MAE loss: 1.35
MAE:  1.2211775
MdAE:  0.8306389
>>> epoch  3
 Average training MAE loss: 2.58
-
 Average eval MAE loss: 2.76
MAE:  2.5130439
MdAE:  1.9953842
>>> epoch  4
 Average training MAE loss: 1.84
-
 Average eval MAE loss: 1.66
MAE:  1.6801354
MdAE:  1.4256918
>>> epoch  5
 Average training MAE loss: 1.89
-
 Average eval MAE loss: 1.56
MAE:  1.3462827
MdAE:  1.2128196
>>> epoch  6
 Average training MAE loss: 2.37
-
 Average eval MAE loss: 2.35
MAE:  2.1457179
MdAE:  2.112069
>>> epoch  7
 Average training MAE loss: 1.85
-
 Average eval MAE loss: 2.66
MAE:  2.29335
MdAE:  2.268663
>>> epoch  8
 Average training MAE loss: 2.18
-
 Average eval MAE loss: 1.42
MAE:  1.2959521
MdAE:  1.0614386
>>> epoch  9
 Average trai

RuntimeError: [enforce fail at inline_container.cc:595] . unexpected pos 492340288 vs 492340176

In [ ]:
pdb off

### Cross Project Training Script - Within Repository

In [ ]:
global WITHIN_PROJECT
WITHIN_PROJECT = False

# within repo
TRAIN_TEST_FILE_PAIRS = [
                        {'train': ['mesos'], 'test': ['usergrid']},
                        {'train': ['usergrid'], 'test': ['mesos']},
                        {'train': ['appceleratorstudio'], 'test': ['aptanastudio']},
                        {'train': ['appceleratorstudio'], 'test': ['titanium']},
                        {'train': ['titanium'], 'test': ['appceleratorstudio']},
                        {'train': ['aptanastudio'], 'test': ['titanium']},
                        {'train': ['mule'], 'test': ['mulestudio']},
                        {'train': ['mulestudio'], 'test': ['mule']}
                        ]


def main():
    global TRAIN_TEST_FILE_PAIRS, MODEL, TOKENIZER, MODEL_NAME
    for file in TRAIN_TEST_FILE_PAIRS:
        if TOKENIZER == 'bbpe':
            config = GPT2Config(num_labels=1, pad_token_id=50257)
        elif TOKENIZER == 'gpt2':
            config = GPT2Config(num_labels=1, pad_token_id=50256)
        elif TOKENIZER == 'wordpiece':
            config = GPT2Config(num_labels=1, pad_token_id=0)
        elif TOKENIZER == 'sentencepiece':
            config = GPT2Config(num_labels=1, pad_token_id=0)
        elif TOKENIZER == 'wordlevel':
            config = GPT2Config(num_labels=1, pad_token_id=3)
        if MODEL_NAME == 'gpt2':
            MODEL = LinearGPT2.from_pretrained('gpt2', config=config)
            MODEL.cuda()
        elif MODEL_NAME == 'gpt2sp':
            MODEL = GPT2SP.from_pretrained('gpt2', config=config)
            MODEL.cuda()
        file_pair, train_dataloader, val_dataloader, all_test_dataloader, test_file_names = data_processing(file_pair=file)
        train_eval_test(file_pair, train_dataloader, val_dataloader, all_test_dataloader, MODEL, test_file_names)
        del MODEL
        torch.cuda.empty_cache()
        global OUTPUT
        with open('./results/' + str(file['train'][0]) + '_' + str(file['test'][0]) +'.txt', 'w+') as f:
            f.writelines(OUTPUT)
            print('results have been written into a text file!')
            OUTPUT = ""


if __name__ == "__main__":
    main()

### Cross Project Training Script - Cross Repository

In [ ]:
global WITHIN_PROJECT
WITHIN_PROJECT = False

# cross repo
TRAIN_TEST_FILE_PAIRS = [
                        {'train': ['clover'], 'test': ['usergrid']},
                        {'train': ['talendesb'], 'test': ['mesos']},
                        {'train': ['talenddataquality'], 'test': ['aptanastudio']},
                        {'train': ['mule'], 'test': ['titanium']},
                        {'train': ['talenddataquality'], 'test': ['appceleratorstudio']},
                        {'train': ['mulestudio'], 'test': ['titanium']},
                        {'train': ['appceleratorstudio'], 'test': ['mulestudio']},
                        {'train': ['appceleratorstudio'], 'test': ['mule']}
                        ]


def main():
    global TRAIN_TEST_FILE_PAIRS, MODEL, TOKENIZER, MODEL_NAME
    for file in TRAIN_TEST_FILE_PAIRS:
        if TOKENIZER == 'gpt2':
            config = GPT2Config(num_labels=1, pad_token_id=50256)
        elif TOKENIZER == 'wordpiece':
            config = GPT2Config(num_labels=1, pad_token_id=0)
        elif TOKENIZER == 'sentencepiece':
            config = GPT2Config(num_labels=1, pad_token_id=0)
        elif TOKENIZER == 'wordlevel':
            config = GPT2Config(num_labels=1, pad_token_id=3)
        if MODEL_NAME == 'gpt2':
            MODEL = LinearGPT2.from_pretrained('gpt2', config=config)
            MODEL.cuda()
        elif MODEL_NAME == 'gpt2sp':
            MODEL = GPT2SP.from_pretrained('gpt2', config=config)
            MODEL.cuda()
        file_pair, train_dataloader, val_dataloader, all_test_dataloader, test_file_names = data_processing(file_pair=file)
        train_eval_test(file_pair, train_dataloader, val_dataloader, all_test_dataloader, MODEL, test_file_names)
        del MODEL
        torch.cuda.empty_cache()
        global OUTPUT
        with open('./results/' + str(file['train'][0]) + '_' + str(file['test'][0]) +'.txt', 'w+') as f:
            f.writelines(OUTPUT)
            print('results have been written into a text file!')
            OUTPUT = ""


if __name__ == "__main__":
    main()